# Education Trust Nasra School - Task Management System

In [ ]:
import time
from functools import wraps
from datetime import datetime

# =============================================================================
#                           HELPER FUNCTIONS
# =============================================================================

def timestamp() -> str:
    """Return a formatted timestamp."""
    return time.strftime("[%Y-%m-%d %H:%M:%S]")

def parse_date(datestr: str) -> datetime:
    """
    Parse a date in 'YYYY-MM-DD' format.
    If invalid/empty, default to today's date.
    """
    if not datestr:
        return datetime.now()
    try:
        return datetime.strptime(datestr, "%Y-%m-%d")
    except ValueError:
        return datetime.now()

# =============================================================================
#                      IN-MEMORY 'DATABASE'
# =============================================================================

"""
USER_DB:
{
  "<username>": {
    "password": str,
    "role": "admin"/"user",
    "name": str,            # Full name
    "is_active": bool,
    "history": [strings of events],
    "tasks_created": [task_id, ...],   # List of tasks user created
    "tasks_assigned": [task_id, ...],  # List of tasks assigned to them
  }, ...
}

TASKS_DB:
{
  "<task_id>": {
    "title": str,
    "description": str,
    "creator": <username>,
    "assignee": <username> or None,
    "due_date": datetime,
    "status": "open"/"in-progress"/"completed"/"closed"/"cancelled",
    "timestamp": str (when created),
    "comments": [
      {"timestamp": "...", "user": "...", "comment": "..."},
      ...
    ]
  },
  ...
}
"""

USER_DB = {
    "admin": {
        "password": "admin123",
        "role": "admin",
        "name": "System Admin",
        "is_active": True,
        "history": ["Admin account created."],
        "tasks_created": [],
        "tasks_assigned": []
    },
    "alice": {
        "password": "alice123",
        "role": "user",
        "name": "Alice Johnson",
        "is_active": True,
        "history": ["Joined as normal user."],
        "tasks_created": [],
        "tasks_assigned": []
    }
}

TASKS_DB = {}
TASK_COUNTER = 1000

current_user = None

# =============================================================================
#                        DECORATORS
# =============================================================================

def login_required(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        global current_user
        if not current_user:
            print("You must be logged in to use this feature.")
            login()
            if not current_user:
                return
        if not USER_DB[current_user]["is_active"]:
            print("Your account is inactive. Contact admin.")
            return
        return func(*args, **kwargs)
    return wrapper

def admin_required(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        global current_user
        if not current_user:
            print("You must be logged in to use this feature.")
            login()
            if not current_user:
                return
        if USER_DB[current_user]["role"] != "admin":
            print("Admin privileges required.")
            return
        return func(*args, **kwargs)
    return wrapper

# =============================================================================
#                         AUTH & SESSION
# =============================================================================

def login():
    global current_user
    uname = input("Enter username: ").strip()
    if uname not in USER_DB:
        print("No such user.")
        return
    if not USER_DB[uname]["is_active"]:
        print("User is inactive. Contact admin.")
        return
    pw = input("Enter password: ").strip()
    if pw == USER_DB[uname]["password"]:
        current_user = uname
        print(f"\nWelcome {USER_DB[uname]['name']}! You are logged in as {USER_DB[uname]['role']}.")
    else:
        print("Incorrect password.")

def logout():
    global current_user
    if not current_user:
        print("No user is currently logged in.")
        return
    print(f"User {USER_DB[current_user]['name']} has been logged out.")
    current_user = None

# =============================================================================
#                         ADMIN FEATURES
# =============================================================================

@admin_required
def create_user():
    """Create a new user (admin/user)."""
    uname = input("Enter new username: ").strip()
    if uname in USER_DB:
        print("That username already exists.")
        return
    role = input("Enter role (admin/user) [default=user]: ").strip() or "user"
    name = input("Enter full name: ").strip()
    pw = input("Set password: ").strip()

    USER_DB[uname] = {
        "password": pw,
        "role": role.lower(),
        "name": name,
        "is_active": True,
        "history": [f"{timestamp()} Created user with role {role}."],
        "tasks_created": [],
        "tasks_assigned": []
    }
    print(f"User '{uname}' created with role '{role}'.")

@admin_required
def toggle_user_status():
    """Activate or deactivate a user."""
    uname = input("Enter username to toggle status: ").strip()
    if uname not in USER_DB:
        print("No such user.")
        return
    if uname == "admin":
        print("Cannot deactivate main admin.")
        return
    current_status = USER_DB[uname]["is_active"]
    USER_DB[uname]["is_active"] = not current_status
    new_status = "Active" if USER_DB[uname]["is_active"] else "Inactive"
    print(f"User '{uname}' is now {new_status}.")

@admin_required
def remove_user():
    """Remove a user from the system."""
    uname = input("Enter username to remove: ").strip()
    if uname not in USER_DB:
        print("No such user.")
        return
    if uname == "admin":
        print("Cannot remove main admin.")
        return
    confirm = input(f"Are you sure to remove '{uname}'? (yes/no): ").strip().lower()
    if confirm != "yes":
        print("Removal cancelled.")
        return
    # Reassign or handle tasks?
    # For now, just remove user from DB
    del USER_DB[uname]
    print(f"User '{uname}' removed from system.")

@admin_required
def view_all_users():
    """View all users and their roles."""
    print("\n=== All Users ===")
    for uname, data in USER_DB.items():
        print(f"Username: {uname}, Role: {data['role']}, Active: {data['is_active']}, Name: {data['name']}")
    print("")

@admin_required
def view_all_tasks_admin():
    """Admin can see all tasks in the system."""
    if not TASKS_DB:
        print("No tasks found.")
        return
    print("\n=== All Tasks ===")
    for tid, tdata in TASKS_DB.items():
        d_str = tdata["due_date"].strftime("%Y-%m-%d") if isinstance(tdata["due_date"], datetime) else tdata["due_date"]
        print(f"TaskID={tid}, Title='{tdata['title']}', Creator={tdata['creator']}, "
              f"Assignee={tdata['assignee']}, Due={d_str}, Status={tdata['status']}")

# =============================================================================
#                     TASK MANAGEMENT (ALL USERS)
# =============================================================================

@login_required
def create_task():
    """User creates a new task (title, description, optional assignee, due date)."""
    global TASK_COUNTER

    title = input("Enter task title: ").strip()
    desc = input("Enter task description: ").strip()
    d_str = input("Enter due date (YYYY-MM-DD) [blank=none]: ").strip()
    due_dt = parse_date(d_str) if d_str else None

    assign = input("Assign to a user? (username or blank=none): ").strip()
    if assign and assign not in USER_DB:
        print("No such user. Will leave unassigned.")
        assign = None

    TASK_COUNTER += 1
    task_id = f"T{TASK_COUNTER}"

    TASKS_DB[task_id] = {
        "title": title,
        "description": desc,
        "creator": current_user,
        "assignee": assign,
        "due_date": due_dt if due_dt else datetime.now(),
        "status": "open",
        "timestamp": timestamp(),
        "comments": []
    }

    # Add to the creator's tasks_created
    USER_DB[current_user]["tasks_created"].append(task_id)
    # If assigned, add to that user's tasks_assigned
    if assign:
        USER_DB[assign]["tasks_assigned"].append(task_id)

    print(f"Task '{title}' created with ID={task_id}.")

@login_required
def view_my_created_tasks():
    """Show tasks created by the current user."""
    my_tasks = USER_DB[current_user]["tasks_created"]
    if not my_tasks:
        print("You have not created any tasks.")
        return
    print("\n=== Tasks You Created ===")
    for tid in my_tasks:
        tdata = TASKS_DB[tid]
        d_str = tdata["due_date"].strftime("%Y-%m-%d") if tdata["due_date"] else "none"
        print(f"ID={tid}, Title='{tdata['title']}', Status={tdata['status']}, Assignee={tdata['assignee']}, Due={d_str}")

@login_required
def view_my_assigned_tasks():
    """Show tasks assigned to the current user."""
    my_assigned = USER_DB[current_user]["tasks_assigned"]
    if not my_assigned:
        print("No tasks assigned to you.")
        return
    print("\n=== Tasks Assigned to You ===")
    for tid in my_assigned:
        tdata = TASKS_DB[tid]
        d_str = tdata["due_date"].strftime("%Y-%m-%d") if tdata["due_date"] else "none"
        print(f"ID={tid}, Title='{tdata['title']}', Status={tdata['status']}, Due={d_str}, Creator={tdata['creator']}")

@login_required
def update_task_status():
    """Change the status of a task if you created it or are assigned to it."""
    tid = input("Enter Task ID to update status: ").strip()
    if tid not in TASKS_DB:
        print("No such task.")
        return
    tdata = TASKS_DB[tid]
    # Check if current user is the creator or assignee
    if tdata["creator"] != current_user and tdata["assignee"] != current_user:
        print("You can only update tasks you created or are assigned to.")
        return
    print(f"Current status: {tdata['status']}. Valid statuses: open, in-progress, completed, closed, cancelled")
    new_status = input("Enter new status: ").strip().lower()
    valid_statuses = ["open", "in-progress", "completed", "closed", "cancelled"]
    if new_status not in valid_statuses:
        print("Invalid status.")
        return
    tdata["status"] = new_status
    print(f"Task {tid} status updated to {new_status}.")

@login_required
def reassign_task():
    """Reassign a task to a different user (only if you're the creator or admin)."""
    global current_user
    tid = input("Enter Task ID to reassign: ").strip()
    if tid not in TASKS_DB:
        print("No such task.")
        return
    tdata = TASKS_DB[tid]
    # If user is admin or is the creator, can reassign
    if USER_DB[current_user]["role"] != "admin" and tdata["creator"] != current_user:
        print("Only the task creator or admin can reassign.")
        return
    new_assignee = input("Enter new assignee (username) or blank to unassign: ").strip()
    if new_assignee and new_assignee not in USER_DB:
        print("No such user.")
        return
    # Remove from old assignee's list if any
    old_assign = tdata["assignee"]
    if old_assign:
        if tid in USER_DB[old_assign]["tasks_assigned"]:
            USER_DB[old_assign]["tasks_assigned"].remove(tid)

    tdata["assignee"] = new_assignee if new_assignee else None
    if new_assignee:
        USER_DB[new_assignee]["tasks_assigned"].append(tid)

    print(f"Task {tid} reassigned to '{new_assignee if new_assignee else 'None'}'.")

@login_required
def add_comment_to_task():
    """Add a comment to a task if you are the creator or assignee."""
    tid = input("Enter Task ID to comment on: ").strip()
    if tid not in TASKS_DB:
        print("No such task.")
        return
    tdata = TASKS_DB[tid]
    if tdata["creator"] != current_user and tdata["assignee"] != current_user and USER_DB[current_user]["role"] != "admin":
        print("Only the creator, assignee, or admin can comment.")
        return
    comment_text = input("Enter your comment: ").strip()
    if not comment_text:
        print("No comment provided.")
        return
    tdata["comments"].append({
        "timestamp": timestamp(),
        "user": current_user,
        "comment": comment_text
    })
    print(f"Comment added to Task {tid}.")

@login_required
def view_task_details():
    """View details of a specific task, including comments, if you can access it."""
    tid = input("Enter Task ID to view: ").strip()
    if tid not in TASKS_DB:
        print("No such task.")
        return
    tdata = TASKS_DB[tid]
    # If user is admin, or creator, or assignee => can view
    if USER_DB[current_user]["role"] != "admin" and tdata["creator"] != current_user and tdata["assignee"] != current_user:
        print("You do not have permission to view this task.")
        return
    d_str = tdata["due_date"].strftime("%Y-%m-%d") if tdata["due_date"] else "none"
    print(f"\n=== Task {tid} Details ===")
    print(f"Title: {tdata['title']}")
    print(f"Description: {tdata['description']}")
    print(f"Creator: {tdata['creator']}, Assignee: {tdata['assignee']}, Status: {tdata['status']}, Due: {d_str}")
    print("Comments:")
    if not tdata["comments"]:
        print("  No comments yet.")
    else:
        for c in tdata["comments"]:
            print(f"  - {c['timestamp']} by {c['user']}: {c['comment']}")

# =============================================================================
#                     TASK FILTER & SEARCH
# =============================================================================

@login_required
def search_tasks():
    """Filter tasks by status, date range, or user involvement."""
    # We'll just demonstrate a simple approach
    st = input("Filter by status (blank=none): ").strip().lower()
    start_str = input("Start due date (YYYY-MM-DD, blank=none): ").strip()
    end_str = input("End due date (YYYY-MM-DD, blank=none): ").strip()
    user_involved = input("User involvement (creator/assignee username or blank=none): ").strip()

    start_dt = parse_date(start_str) if start_str else None
    end_dt = parse_date(end_str) if end_str else None

    results = []
    for tid, tdata in TASKS_DB.items():
        # status filter
        if st and tdata["status"] != st:
            continue
        # date range filter
        due = tdata["due_date"] if tdata["due_date"] else datetime.now()
        if start_dt and due < start_dt:
            continue
        if end_dt and due > end_dt:
            continue
        # user involvement
        if user_involved:
            if tdata["creator"] != user_involved and tdata["assignee"] != user_involved:
                continue
        # If user is not admin, we must ensure they only see tasks they are allowed to see
        if USER_DB[current_user]["role"] != "admin":
            # Must be either creator or assignee
            if tdata["creator"] != current_user and tdata["assignee"] != current_user:
                continue
        results.append((tid, tdata))

    if not results:
        print("No tasks match your filters.")
        return

    print("\n=== Filtered Task Results ===")
    for tid, tdata in results:
        d_str = tdata["due_date"].strftime("%Y-%m-%d") if tdata["due_date"] else "none"
        print(f"ID={tid}, Title='{tdata['title']}', Status={tdata['status']}, Creator={tdata['creator']}, "
              f"Assignee={tdata['assignee']}, Due={d_str}")

# =============================================================================
#                         MAIN MENU
# =============================================================================

def main_menu():
    while True:
        print("Education Trust Nasra School - Task Management System")
        print(f"Current User: {current_user if current_user else 'None'}")
        print("--------------------------------------------")
        print("1.  Login")
        print("2.  Logout")

        print("\n-- Admin Features --")
        print("3.  Create User")
        print("4.  Toggle User Status")
        print("5.  Remove User")
        print("6.  View All Users")
        print("7.  View All Tasks (Admin)")

        print("\n-- Task Management (All Users) --")
        print("8.  Create Task")
        print("9.  View My Created Tasks")
        print("10. View My Assigned Tasks")
        print("11. Update Task Status")
        print("12. Reassign Task (if creator or admin)")
        print("13. Add Comment to Task")
        print("14. View Task Details")

        print("\n-- Task Filtering/Search --")
        print("15. Search/Filter Tasks")

        print("\n16. Exit")

        choice = input("Enter your choice: ").strip()
        if choice == "1":
            login()
        elif choice == "2":
            logout()
        elif choice == "3":
            create_user()
        elif choice == "4":
            toggle_user_status()
        elif choice == "5":
            remove_user()
        elif choice == "6":
            view_all_users()
        elif choice == "7":
            view_all_tasks_admin()
        elif choice == "8":
            create_task()
        elif choice == "9":
            view_my_created_tasks()
        elif choice == "10":
            view_my_assigned_tasks()
        elif choice == "11":
            update_task_status()
        elif choice == "12":
            reassign_task()
        elif choice == "13":
            add_comment_to_task()
        elif choice == "14":
            view_task_details()
        elif choice == "15":
            search_tasks()
        elif choice == "16":
            print("\nExiting Education Trust Nasra School - Task Management System")
            break
        else:
            print("Invalid choice. Please try again.")

# =============================================================================
#                     SCRIPT ENTRY POINT
# =============================================================================

if __name__ == "__main__":
    main_menu()


Education Trust Nasra School - Task Management System
Current User: None
--------------------------------------------
1.  Login
2.  Logout

-- Admin Features --
3.  Create User
4.  Toggle User Status
5.  Remove User
6.  View All Users
7.  View All Tasks (Admin)

-- Task Management (All Users) --
8.  Create Task
9.  View My Created Tasks
10. View My Assigned Tasks
11. Update Task Status
12. Reassign Task (if creator or admin)
13. Add Comment to Task
14. View Task Details

-- Task Filtering/Search --
15. Search/Filter Tasks

16. Exit
Enter your choice: 
Invalid choice. Please try again.
Education Trust Nasra School - Task Management System
Current User: None
--------------------------------------------
1.  Login
2.  Logout

-- Admin Features --
3.  Create User
4.  Toggle User Status
5.  Remove User
6.  View All Users
7.  View All Tasks (Admin)

-- Task Management (All Users) --
8.  Create Task
9.  View My Created Tasks
10. View My Assigned Tasks
11. Update Task Status
12. Reassign Task (